### 什么是工具（Tools）
AI Agents能够根据任务需要，灵活地连接并使用外部系统的功能，从而极大地拓展了大语言模型的能力边界。Tools的诞生，正是源于这种让AI Agents与API、数据库、文件系统等外部系统直接进行交互的需求。
为了实现这个目的，LangChain允许我们将可调用的函数（callable function）以及对应的输入模式（input schema）封装成“Tools”。而通过使用Tools，大模型能够按照已经事先定义好的输入输出格式，去整理自己从用户处接收，或者向外部系统输出的信息，从而更好地满足外部系统的接入要求。
### 如何创建Tools
#### 1.使用@tool装饰器，并定义函数
使用@tool装饰器，我们可以将一个普通的Python函数转换为一个LangChain Tool。
在定义函数时，必须要写明类型提示（type hints）。
#### 2.利用函数的文档字符串进行工具描述
我们需要使用文档字符串，对工具的用途，参数的含义进行清晰、简明的描述，以帮助模型或Agent理解使用这个工具的目的。
#### 3.编写函数体，设定工具的业务逻辑
完成函数体的内容，写明工具的具体业务逻辑。
```python
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"
```
### 自定义工具属性
#### 1.自定义工具名称
默认情况下，工具名称来源于函数名称。可以覆盖
```python
@tool("web_search")  # Custom name
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search
```
#### 2.自定义工具描述
覆盖自动生成的工具模型，更好地引导模型的使用
```python
@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems.")
def calc(expression: str) -> str:
    """Evaluate mathematical expressions."""
    return str(eval(expression))
```
### 高级模式定义
为了约束外部输入数据的结构，让Tools封装的参数输入模式更为精准，LangChain允许我们在@tool后面的括号种设定自定义的参数模式。
```python
@tool("search", args_schema=SearchInputSchema)
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"
```
而关于SearchInputSchema的具体设定方式，有两种方案
#### 1.Pydantic model
使用Pydantic定义一个数据模型，作为工具的输入模式。
```python
from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )

@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result
```
#### 2.字典格式
直接使用字典定义参数模式，适用于简单的场景。
```python
weather_schema = {
    "type": "object",
    "properties": {
        "location": {"type": "string"},
        "units": {"type": "string"},
        "include_forecast": {"type": "boolean"}
    },
    "required": ["location", "units", "include_forecast"]
}

@tool(args_schema=weather_schema)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result
```
